In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import catboost as cb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/test.csv")
print(train.shape)
train.head()

In [ ]:
print(train.isnull().sum())
print("____________________")
print(test.isnull().sum())

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data = train.corr(), annot = True, cmap = 'Blues')
plt.show()

In [ ]:
train['u_in_cumsum'] = (train['u_in']).groupby(train['breath_id']).cumsum()

test['u_in_cumsum'] = (test['u_in']).groupby(test['breath_id']).cumsum()
train['u_in_lag'] = train['u_in'].shift(2)
train = train.fillna(0)

test['u_in_lag'] = test['u_in'].shift(2)
test = test.fillna(0)


In [ ]:

train_fin = train.drop("id",axis=1)
train_fin.shape

In [ ]:
train_fin.isnull().sum()

In [ ]:
train_fin = train_fin.sample(frac =.003,random_state = 90, replace = False)
train_fin.shape

# Modelling

In [ ]:
x = train_fin.drop("pressure",axis=1)
y = train_fin["pressure"]
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor().fit(x, y)
pred_gbr = gbr.predict(x)
mean_absolute_error(y, pred_gbr)

In [ ]:
x.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers,callbacks
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# early_stopping = callbacks.EarlyStopping(
#     min_delta=0.001, 
#     patience=20, 
#     restore_best_weights=True,
# )
# neuralnet = keras.Sequential([
#     layers.Dense(300,activation="relu",input_shape = [8]),
#     layers.Dense(512,activation="relu"),
#     layers.Dense(387,activation="relu"),
#     layers.Dense(234,activation="relu"),
#     layers.Dense(104,activation="relu"),
#     layers.Dense(56,activation="relu"),
#     layers.Dense(20,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(11,activation="relu"),
#     layers.Dense(1),
# ])

In [ ]:
# neuralnet.compile(
#     optimizer='adam',
#     loss='cross-entropy',
# )

In [ ]:
# history = neuralnet.fit(
#     x, y,
#     validation_data=(x, y),
#     batch_size=32,
#     epochs=50
# )

In [ ]:
x_test = test.drop("id",axis=1)
# test_pred = neuralnet.predict(
#     x_test,
#     batch_size=32,
#     verbose = 1
# )

In [ ]:
train_dataset = cb.Pool(x, y) 

In [ ]:
cbmodel = cb.CatBoostRegressor(loss_function='RMSE')
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
cbmodel.grid_search(grid, train_dataset)
pred = cbmodel.predict(x_test)

In [ ]:
# print(test_pred)
# result = test_pred.flatten()

test_gbr = gbr.predict(x_test)
output = pd.DataFrame({'id': test.id, 'pressure': pred})
output.to_csv('submission.csv', index=False)